In [4]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import re

# Tentukan path folder
source_folder = 'hasil scraping'
output_folder = 'cleaning_data'

# Cek dan buat folder sumber jika belum ada
if not os.path.exists(source_folder):
    print(f"❌ Folder '{source_folder}' tidak ditemukan!")
    print("Pastikan folder 'hasil scraping' dengan dataset sudah tersedia.")
    exit()
else:
    print(f"✓ Folder sumber ditemukan: {source_folder}")

# Cek dan buat folder output secara otomatis
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"✓ Folder '{output_folder}' berhasil dibuat")
else:
    print(f"✓ Folder '{output_folder}' sudah ada")

print(f"✓ Membaca dataset dari folder: {source_folder}")
print(f"✓ Hasil cleaning akan disimpan ke folder: {output_folder}")

# Dictionary untuk menyimpan semua dataset
datasets = {}

print("\n" + "="*60)
print("LOADING DATASETS DARI FOLDER HASIL SCRAPING")
print("="*60)

# Mapping file yang akan dibaca
file_mappings = {
    'alun_alun': 'alun_alun_kota_wisata_batu_reviews_20250725_105504.csv',
    'bns': 'batu_night_spectacular_reviews_with_visit_time_20250724_154759.csv',
    'coban_talun': 'coban_talun_reviews_with_visit_time_20250724_114716.csv',
    'eco': 'eco_active_park_reviews_with_visit_time_20250725_002543.csv',
    'panderman': 'gunung_panderman_reviews_20250724_141423.csv',
    'jp1': 'jatim_park_1_reviews_with_visit_time_20250723_140636.csv',
    'jp2': 'jatim_park_2_reviews_with_visit_time_20250725_114708.csv',
    'jp3': 'jatim_park_3_reviews_with_visit_time_20250725_011845.csv',
    'museum': 'museum_angkut_reviews_with_visit_time_20250725_160008.csv',
    'selecta': 'taman_selecta_reviews_with_visit_time_20250724_085149.csv'
}

# Load semua dataset
for key, filename in file_mappings.items():
    file_path = os.path.join(source_folder, filename)
    try:
        df = pd.read_csv(file_path)
        datasets[key] = df
        print(f"✓ {filename} berhasil dimuat ({len(df)} rows)")
    except FileNotFoundError:
        print(f"❌ File tidak ditemukan: {filename}")
    except Exception as e:
        print(f"❌ Error loading {filename}: {e}")

if len(datasets) == 0:
    print("❌ Tidak ada dataset yang berhasil dimuat!")
    exit()

print(f"\n✓ Total dataset berhasil dimuat: {len(datasets)}")

# Fungsi untuk membersihkan dan standarisasi data
def clean_dataset(df, dataset_name):
    """
    Fungsi untuk membersihkan dataset review wisata
    """
    print(f"\n{'='*50}")
    print(f"Cleaning dataset: {dataset_name}")
    print(f"{'='*50}")
    
    # Informasi awal
    print(f"Jumlah data awal: {len(df)}")
    print(f"Kolom: {list(df.columns)}")
    
    # 1. Hapus duplikat
    df_clean = df.drop_duplicates()
    duplicates_removed = len(df) - len(df_clean)
    if duplicates_removed > 0:
        print(f"✓ Duplikat dihapus: {duplicates_removed}")
    
    # 2. Tangani missing values
    missing_before = df_clean.isnull().sum().sum()
    
    # Untuk kolom teks, isi dengan string kosong
    text_columns = ['reviewer_name', 'review_text']
    for col in text_columns:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].fillna('')
    
    # 3. Bersihkan dan standarisasi visit_time - HANYA 4 KATEGORI
    if 'visit_time' in df_clean.columns:
        def clean_visit_time(visit_time):
            if pd.isna(visit_time) or visit_time == '' or str(visit_time).strip() == '':
                return 'Tidak diketahui'
            
            visit_time_str = str(visit_time).strip().lower()
            
            # HANYA 3 kategori utama yang diizinkan
            if visit_time_str in ['hari biasa', 'weekday']:
                return 'Hari biasa'
            elif visit_time_str in ['akhir pekan', 'weekend', 'akhir p', 'akhir pekan…', 'akhir p…']:
                return 'Akhir pekan'
            elif visit_time_str in ['hari libur nasional', 'libur nasional', 'hari libur nas', 'hari libur nasional…', 'hari libur nas…']:
                return 'Hari libur nasional'
            else:
                # Semua yang lain menjadi 'Tidak diketahui'
                return 'Tidak diketahui'
        
        # Tampilkan sample visit_time sebelum cleaning
        print(f"   Sample visit_time sebelum cleaning:")
        unique_before = df_clean['visit_time'].dropna().unique()[:5]
        for sample in unique_before:
            print(f"     '{sample}'")
        
        # Apply cleaning function
        df_clean['visit_time'] = df_clean['visit_time'].apply(clean_visit_time)
        print(f"✓ Visit time distandardisasi ke 3 kategori + Tidak diketahui")
        
        # Tampilkan distribusi visit_time setelah cleaning
        visit_time_dist = df_clean['visit_time'].value_counts()
        print(f"   Distribusi visit_time setelah cleaning:")
        for visit_type, count in visit_time_dist.items():
            percentage = (count / len(df_clean)) * 100
            print(f"     '{visit_type}': {count} ({percentage:.1f}%)")
    
    missing_after = df_clean.isnull().sum().sum()
    if missing_before > 0:
        print(f"✓ Missing values ditangani: {missing_before} → {missing_after}")
    
    # 4. Standarisasi rating (pastikan integer 1-5)
    if 'rating' in df_clean.columns:
        df_clean['rating'] = pd.to_numeric(df_clean['rating'], errors='coerce')
        df_clean['rating'] = df_clean['rating'].fillna(0).astype(int)
        df_clean['rating'] = df_clean['rating'].clip(1, 5)  # Batasi antara 1-5
        print(f"✓ Rating distandardisasi (1-5)")
    
    # 5. Bersihkan text dari karakter khusus berlebihan
    def clean_text(text):
        if pd.isna(text) or text == '':
            return ''
        # Hapus multiple spaces
        text = re.sub(r'\s+', ' ', str(text))
        # Hapus leading/trailing spaces
        text = text.strip()
        return text
    
    for col in text_columns:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].apply(clean_text)
    
    # 6. Standarisasi date format
    if 'date' in df_clean.columns:
        # Konversi date relative ke format standar
        def standardize_date(date_str):
            if pd.isna(date_str):
                return 'Tidak diketahui'
            
            date_str = str(date_str).lower()
            
            # Mapping untuk konversi
            if 'hari lalu' in date_str:
                return 'Beberapa hari lalu'
            elif 'minggu lalu' in date_str or 'seminggu lalu' in date_str:
                return '1 minggu lalu'
            elif 'bulan lalu' in date_str or 'sebulan lalu' in date_str:
                num = re.search(r'(\d+)', date_str)
                if num:
                    return f'{num.group(1)} bulan lalu'
                return '1 bulan lalu'
            elif 'tahun lalu' in date_str or 'setahun lalu' in date_str:
                num = re.search(r'(\d+)', date_str)
                if num:
                    return f'{num.group(1)} tahun lalu'
                return '1 tahun lalu'
            else:
                return date_str
        
        df_clean['date'] = df_clean['date'].apply(standardize_date)
    
    # 7. Filter data yang tidak valid
    # Hapus review dengan text kosong atau terlalu pendek
    if 'review_text' in df_clean.columns:
        before_filter = len(df_clean)
        df_clean = df_clean[df_clean['review_text'].str.len() > 10]
        filtered = before_filter - len(df_clean)
        if filtered > 0:
            print(f"✓ Review terlalu pendek dihapus: {filtered}")
    
    # 8. Tambahkan kolom wisata
    df_clean['wisata'] = dataset_name
    
    print(f"\nJumlah data setelah cleaning: {len(df_clean)}")
    if len(df) > 0:
        print(f"Persentase data yang tersisa: {len(df_clean)/len(df)*100:.1f}%")
    
    return df_clean

# Mapping nama dataset ke nama wisata yang lebih readable
wisata_names = {
    'alun_alun': 'Alun Alun Kota Wisata Batu',
    'bns': 'Batu Night Spectacular',
    'coban_talun': 'Coban Talun',
    'eco': 'Eco Active Park',
    'panderman': 'Gunung Panderman',
    'jp1': 'Jatim Park 1',
    'jp2': 'Jatim Park 2',
    'jp3': 'Jatim Park 3',
    'museum': 'Museum Angkut',
    'selecta': 'Taman Selecta'
}

print("\n" + "="*60)
print("PROSES CLEANING DATASETS")
print("="*60)

# Clean semua dataset
cleaned_datasets = {}
for key, df in datasets.items():
    cleaned_df = clean_dataset(df, wisata_names[key])
    cleaned_datasets[key] = cleaned_df

# Gabungkan semua dataset yang sudah dibersihkan
if cleaned_datasets:
    all_reviews = pd.concat(cleaned_datasets.values(), ignore_index=True)
    
    print("\n" + "="*60)
    print("RINGKASAN DATASET GABUNGAN")
    print("="*60)
    print(f"Total review: {len(all_reviews)}")
    print(f"\nDistribusi review per wisata:")
    print(all_reviews['wisata'].value_counts())

    print(f"\nDistribusi rating:")
    print(all_reviews['rating'].value_counts().sort_index())

    print(f"\nRata-rata rating per wisata:")
    avg_ratings = all_reviews.groupby('wisata')['rating'].agg(['mean', 'count']).round(2)
    avg_ratings = avg_ratings.sort_values('mean', ascending=False)
    print(avg_ratings)

    # Cek kolom yang ada
    print(f"\nKolom yang tersedia:")
    print(all_reviews.columns.tolist())

    # Statistik visit_time - hanya 4 kategori
    if 'visit_time' in all_reviews.columns:
        print(f"\nDistribusi waktu kunjungan (4 kategori saja):")
        visit_dist = all_reviews['visit_time'].value_counts()
        for visit_type, count in visit_dist.items():
            percentage = (count / len(all_reviews)) * 100
            print(f"  '{visit_type}': {count} ({percentage:.1f}%)")
        
        # Validasi bahwa hanya ada 4 kategori - dengan penanganan NaN
        unique_categories = all_reviews['visit_time'].dropna().unique()  # Hapus NaN sebelum sorting
        expected_categories = ['Hari biasa', 'Akhir pekan', 'Hari libur nasional', 'Tidak diketahui']
        print(f"\nValidasi kategori visit_time:")
        print(f"  Kategori yang ada: {sorted(unique_categories)}")
        print(f"  Kategori yang diharapkan: {sorted(expected_categories)}")
        print(f"  ✓ Sesuai ekspektasi: {set(unique_categories) == set(expected_categories)}")
        
        # Cek apakah ada NaN yang tersisa
        nan_count = all_reviews['visit_time'].isna().sum()
        if nan_count > 0:
            print(f"  ⚠️  Masih ada {nan_count} nilai NaN yang perlu ditangani")

    print("\n" + "="*60)
    print("MENYIMPAN DATASET KE FOLDER CLEANING_DATA")
    print("="*60)

    # Pastikan tidak ada NaN di visit_time sebelum menyimpan
    if 'visit_time' in all_reviews.columns:
        all_reviews['visit_time'] = all_reviews['visit_time'].fillna('Tidak diketahui')
        print("✓ NaN di visit_time diubah menjadi 'Tidak diketahui'")

    # Simpan dataset gabungan yang sudah bersih
    combined_filename = os.path.join(output_folder, 'combined_batu_tourism_reviews_cleaned.csv')
    all_reviews.to_csv(combined_filename, index=False)
    print(f"✓ Dataset gabungan disimpan: {combined_filename}")

    # Simpan juga per wisata
    for key, df in cleaned_datasets.items():
        # Pastikan tidak ada NaN di setiap dataset individual
        if 'visit_time' in df.columns:
            df['visit_time'] = df['visit_time'].fillna('Tidak diketahui')
        
        filename = os.path.join(output_folder, f'{key}_reviews_cleaned.csv')
        df.to_csv(filename, index=False)
        print(f"✓ {wisata_names[key]} disimpan: {filename}")

    # Buat laporan kualitas data
    def data_quality_report(df):
        """
        Generate laporan kualitas data
        """
        report = []
        report.append("="*60)
        report.append("LAPORAN KUALITAS DATA - BATU TOURISM REVIEWS")
        report.append("="*60)
        report.append(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        report.append(f"Total reviews: {len(df)}")
        
        # 1. Completeness
        report.append("\n1. KELENGKAPAN DATA:")
        for col in df.columns:
            missing_pct = (df[col].isnull().sum() / len(df)) * 100
            empty_pct = ((df[col] == '').sum() / len(df)) * 100
            report.append(f"   {col}: {100 - missing_pct - empty_pct:.1f}% lengkap")
        
        # 2. Consistency
        report.append("\n2. KONSISTENSI DATA:")
        report.append(f"   Rating dalam range 1-5: {((df['rating'] >= 1) & (df['rating'] <= 5)).all()}")
        
        if 'visit_time' in df.columns:
            unique_visit_times = df['visit_time'].nunique()
            report.append(f"   Nilai unik visit_time: {unique_visit_times}")
            # Hapus NaN sebelum sorting untuk laporan
            categories = [cat for cat in df['visit_time'].unique() if pd.notna(cat)]
            report.append(f"   Kategori visit_time: {sorted(categories)}")
        
        # 3. Data distribution
        report.append("\n3. DISTRIBUSI DATA:")
        report.append(f"   Rata-rata panjang review: {df['review_text'].str.len().mean():.0f} karakter")
        report.append(f"   Review terpendek: {df['review_text'].str.len().min()} karakter")
        report.append(f"   Review terpanjang: {df['review_text'].str.len().max()} karakter")
        
        # 4. Visit time distribution (hanya 4 kategori)
        if 'visit_time' in df.columns:
            report.append("\n4. DISTRIBUSI WAKTU KUNJUNGAN:")
            visit_counts = df['visit_time'].value_counts()
            for visit_type in ['Hari biasa', 'Akhir pekan', 'Hari libur nasional', 'Tidak diketahui']:
                count = visit_counts.get(visit_type, 0)
                percentage = (count / len(df)) * 100
                report.append(f"   '{visit_type}': {count} ({percentage:.1f}%)")
        
        # 5. Rating distribution
        report.append("\n5. DISTRIBUSI RATING:")
        rating_counts = df['rating'].value_counts().sort_index()
        for rating, count in rating_counts.items():
            percentage = (count / len(df)) * 100
            report.append(f"   Rating {rating}: {count} ({percentage:.1f}%)")
        
        # 6. Potential issues
        report.append("\n6. POTENSI MASALAH:")
        short_reviews = (df['review_text'].str.len() < 50).sum()
        report.append(f"   Review < 50 karakter: {short_reviews} ({short_reviews/len(df)*100:.1f}%)")
        
        suspicious_names = df['reviewer_name'].str.match(r'^[a-zA-Z0-9_]+$').sum()
        report.append(f"   Nama reviewer hanya alfanumerik: {suspicious_names}")
        
        # 7. Statistik per wisata
        report.append("\n7. STATISTIK PER WISATA:")
        for wisata in sorted(df['wisata'].unique()):
            wisata_data = df[df['wisata'] == wisata]
            count = len(wisata_data)
            avg_rating = wisata_data['rating'].mean()
            avg_length = wisata_data['review_text'].str.len().mean()
            
            report.append(f"   {wisata}:")
            report.append(f"     - Total reviews: {count}")
            report.append(f"     - Avg rating: {avg_rating:.2f}")
            report.append(f"     - Avg review length: {avg_length:.0f} chars")
            
            if 'visit_time' in df.columns:
                visit_counts = wisata_data['visit_time'].value_counts()
                visit_info = {}
                for cat in ['Hari biasa', 'Akhir pekan', 'Hari libur nasional', 'Tidak diketahui']:
                    if cat in visit_counts:
                        visit_info[cat] = visit_counts[cat]
                report.append(f"     - Visit time distribution: {visit_info}")
        
        return '\n'.join(report)

    # Generate dan simpan laporan kualitas
    quality_report = data_quality_report(all_reviews)
    print("\n" + quality_report)

    # Simpan laporan ke file
    report_filename = os.path.join(output_folder, 'data_quality_report.txt')
    with open(report_filename, 'w', encoding='utf-8') as f:
        f.write(quality_report)
    print(f"\n✓ Laporan kualitas data disimpan: {report_filename}")

    # Buat summary statistics dan simpan
    summary_data = []
    for wisata in sorted(all_reviews['wisata'].unique()):
        wisata_data = all_reviews[all_reviews['wisata'] == wisata]
        
        summary_row = {
            'Wisata': wisata,
            'Total_Reviews': len(wisata_data),
            'Avg_Rating': round(wisata_data['rating'].mean(), 2),
            'Avg_Review_Length': round(wisata_data['review_text'].str.len().mean(), 0)
        }
        
        # Tambahkan distribusi visit_time (4 kategori saja)
        if 'visit_time' in wisata_data.columns:
            visit_counts = wisata_data['visit_time'].value_counts()
            summary_row['Hari_Biasa'] = visit_counts.get('Hari biasa', 0)
            summary_row['Akhir_Pekan'] = visit_counts.get('Akhir pekan', 0)
            summary_row['Hari_Libur_Nasional'] = visit_counts.get('Hari libur nasional', 0)
            summary_row['Tidak_Diketahui'] = visit_counts.get('Tidak diketahui', 0)
        
        summary_data.append(summary_row)
    
    summary_stats = pd.DataFrame(summary_data)
    summary_filename = os.path.join(output_folder, 'summary_statistics.csv')
    summary_stats.to_csv(summary_filename, index=False)
    print(f"✓ Summary statistik disimpan: {summary_filename}")

    # Tampilkan daftar file yang telah disimpan
    print(f"\n" + "="*60)
    print("DAFTAR FILE YANG DISIMPAN DI FOLDER CLEANING_DATA")
    print("="*60)
    
    try:
        files_in_folder = [f for f in os.listdir(output_folder) if f.endswith('.csv') or f.endswith('.txt')]
        files_in_folder.sort()
        
        for i, file in enumerate(files_in_folder, 1):
            file_path = os.path.join(output_folder, file)
            if os.path.exists(file_path):
                file_size = os.path.getsize(file_path) / 1024  # KB
                print(f"{i:2d}. {file} ({file_size:.1f} KB)")

        print(f"\n✓ Total {len(files_in_folder)} file berhasil disimpan di folder '{output_folder}'")
        
    except Exception as e:
        print(f"❌ Error saat membaca folder output: {e}")
    
else:
    print("❌ Tidak ada dataset yang berhasil dimuat dan dibersihkan!")

print(f"\n{'='*60}")
print("PROSES CLEANING SELESAI")
print(f"{'='*60}")
print(f"✓ Folder output: {output_folder}")
print(f"✓ Dataset gabungan: combined_batu_tourism_reviews_cleaned.csv")
print(f"✓ Laporan kualitas: data_quality_report.txt")
print(f"✓ Summary statistik: summary_statistics.csv")
print(f"✓ Visit time categories: HANYA 4 kategori (Hari biasa, Akhir pekan, Hari libur nasional, Tidak diketahui)")

✓ Folder sumber ditemukan: hasil scraping
✓ Folder 'cleaning_data' sudah ada
✓ Membaca dataset dari folder: hasil scraping
✓ Hasil cleaning akan disimpan ke folder: cleaning_data

LOADING DATASETS DARI FOLDER HASIL SCRAPING
✓ alun_alun_kota_wisata_batu_reviews_20250725_105504.csv berhasil dimuat (759 rows)
✓ batu_night_spectacular_reviews_with_visit_time_20250724_154759.csv berhasil dimuat (328 rows)
✓ coban_talun_reviews_with_visit_time_20250724_114716.csv berhasil dimuat (286 rows)
✓ eco_active_park_reviews_with_visit_time_20250725_002543.csv berhasil dimuat (913 rows)
✓ gunung_panderman_reviews_20250724_141423.csv berhasil dimuat (364 rows)
✓ jatim_park_1_reviews_with_visit_time_20250723_140636.csv berhasil dimuat (414 rows)
✓ jatim_park_2_reviews_with_visit_time_20250725_114708.csv berhasil dimuat (756 rows)
✓ jatim_park_3_reviews_with_visit_time_20250725_011845.csv berhasil dimuat (256 rows)
✓ museum_angkut_reviews_with_visit_time_20250725_160008.csv berhasil dimuat (1243 rows)
✓ 